# Let

### A Local, Temporary Place for Your Stuff

In [1]:
;; A simple function that computes the discount for books,
;; but with a minimum charge that overrides the discount

(defn compute-discount-amount [amount discount-percent min-charge]
    (if (> (* amount (- 1.0 discount-percent)) min-charge)
        (* amount (- 1.0 discount-percent))
        min-charge))


#'user/compute-discount-amount

In [2]:
;; Using 'def' to store a local binding to a value (not recommended)

(defn compute-discount-amount [amount discount-percent min-charge]
    (def discounted-amount (* amount (- 1.0 discount-percent))) ; NOOOOO!
        (if (> discounted-amount min-charge)
            discounted-amount
            min-charge))


#'user/compute-discount-amount

In [3]:
;; Symbols bounded with 'def' have global visibility, 
;; so any new bindings outside the function can have unintended consequences

(def discounted-amount "Some random string.")

(compute-discount-amount 10.0 0.20 1.0)


8.0

In [7]:
;; Using 'let' to store a local binding to a value (recommended)

(defn compute-discount-amount [amount discount-percent min-charge]
    (let [discounted-amount (* amount (- 1.0 discount-percent))]
        (if (> discounted-amount min-charge)
            discounted-amount
            min-charge)))

(compute-discount-amount 10.0 0.20 15.0)


15.0

In [8]:
;; The 'let' expression inside the last function 
;; can be divided for further clarity

(defn compute-discount-amount [amount discount-percent min-charge]
    (let [discount (* amount discount-percent)
          discounted-amount (- amount discount)]
        (if (> discounted-amount min-charge)
            discounted-amount
            min-charge)))

(compute-discount-amount 10.0 0.20 20.0)



20.0

let over fn

In [12]:
;; Given a map of user discounts

(def user-discounts {"Nicholas" 0.10 "Jonathan" 0.07 "Felicia" 0.05})

#'user/user-discounts

In [24]:
;; A function can be created to compute the discount 
;; based on the last 'user-discounts' map, as follows

(defn compute-discount-amount [amount user-name user-discounts min-charge]
    (let [discount-percent (user-discounts user-name)
          discount (* amount discount-percent)
          discounted-amount (- amount discount)]
        (if (> discounted-amount min-charge)
            discounted-amount
            min-charge)))

(compute-discount-amount 10.0 "Felicia" user-discounts 1.0)


9.5

In [27]:
;; The last function can be converted to a higher order function that creates 
;; variants of 'compute-discount-amount' tailored to a particular customer

(defn mk-discount-price-f [user-name user-discounts min-charge]
    (let [discount-percent (user-discounts user-name)]
        (fn [amount]
            (let [discount (* amount discount-percent)
                  discounted-amount (- amount discount)]
                (if (> discounted-amount min-charge)
                    discounted-amount
                    min-charge)))))

;; The price function for Felicia

(def compute-felicia-price (mk-discount-price-f "Felicia" user-discounts 10.0))
(compute-felicia-price 20.0)


19.0

### Variations on the theme

In [29]:
;; Given the following maps

(def anonymous-book {:title "Sir Gawain and the Green Knight"})

(def with-author {:title "Once and Future King" :author "White"})


#'user/with-author

In [30]:
;; A function that returns the uppercase name of the author
;; (or nil if there's no author) can be created as follows

(defn uppercase-author [book]
    (let [author (:author book)]
        (if author
            (.toUpperCase author))))


#'user/uppercase-author

In [32]:
(uppercase-author with-author)

"WHITE"

In [31]:
(uppercase-author anonymous-book)

nil

In [36]:
;;The last function can be shortened with 'if-let', as follows

(defn uppercase-author [book]
    (if-let [author (:author book)]
        (.toUpperCase author)
        "ANONYMOUS"))

#'user/uppercase-author

In [37]:
(uppercase-author with-author)

"WHITE"

In [38]:
(uppercase-author anonymous-book)

"ANONYMOUS"

In [42]:
;; There's also a 'when-let' macro, which can be used as follows

(defn uppercase-author [book]
    (when-let [author (:author book)]
        (.toUpperCase author)))


#'user/uppercase-author

In [43]:
(uppercase-author with-author)

"WHITE"

In [44]:
(uppercase-author anonymous-book)

nil

### Issues with let

In [47]:
;; We can use title inside of the let scope.

(let [title "Let's Pretend This Never Happened"]
    (println "The title is" title))


The title is Let's Pretend This Never Happened


nil

In [49]:
;; But now we're outside of the let scope, so an exception is raised.

(defn print-the-title []
    (println "The title is" title)) ; Boom!

Syntax error compiling at (REPL:4:5).
Unable to resolve symbol: title in this context


class clojure.lang.Compiler$CompilerException: 

In [50]:
;; A binding in a let can shadow a binding in an outer let

(let [title "Pride and Prejudice"]
    (let [title "Sense and Sensibility"]
        (println title))) 

Sense and Sensibility


nil

In [51]:
; A binding in a let can also be overriden with
;; another binding inside of the same let

(let [title "Pride and Prejudice"
      title (str title " and Zombies")]
    (println title))

Pride and Prejudice and Zombies


nil